# Fluent Python
https://github.com/fluentpython/example-code

파이썬 용어집 

https://docs.python.org/ko/3/glossary.html

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np


## CHAPTER 17. Concurrency with Futures

### Downloading with concurrent.futures

concurrent.futures package are the ThreadPoolExecutor and ProcessPoolExecutor classes

콜러블객체를 서로 다른 스레드나 프로세스에서 실행할수 잇게 해주는 인터페이스를 구현 

작업자 스레드나 작업자 프로세스를 관리하는 풀과 실행할 작업을 담은 큐를 가짐 


In [3]:
def download_many(cc_list):
    # 작업할 스레드 최소 개수 
    workers = min(MAX_WORKERS, len(cc_list))
    # 작업자 수를 전달해서 Thread 객체 생성 
    # executor.__exit__() 가 executor.shutdown(wait=True) 호출 
    # 모든 스레드 완료될때까지 블록됨
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(download_one, sorted(cc_list)) # 여러 스레드 호출. 결과값만 반환함
    return len(list(res))

### Where Are the Futures?

Future 클래스 
1. concurrent.futrues.Future
2. asyncio.Future 

두 객체 지연된 계산을 표현하기 위해 사용 

객체를 직접 생성하면 안된다. 

Future 객체는 concurrent.futures 나 asyncio 같은 동시성 프레임워크에서만 배타적으로 생성해야 함.

Future 는 앞으로 일어날 일을 나타내고, Future 의 실행을 스케줄링하는 프레임워크만이 어떤 일이 일어날지 확실히 알수 있음

current.futures.Future 객체는 concurrent.futures.Executor 의 서브클래스로 실행을 스케쥴링 한 후에만 생성 됨 

Executor.submit() 콜러블을 받아서 콜러블의 실행을 스케쥴링하고 Future 객체를 반환

Future 클래스 객체에 연결된 콜러블 실행의 완료 여부를 done() 메서드를 통해 전달 받거나 add_done_callback() 메서드를 통해 통보요청함 

result() 메서드는 완료된 경우 콜러블 결과를 반환 또는 예외를 다시 발생시킴

그러나 Future 객체 실행이 완료되지 않은경우 result() 동작이 다름 
1. concurrency.futures.Future: f.result() 결과가 나올때까지 호출자의 스레드를 블로킹함. timeout 인수 전달 가능 
2. asyncio.Future: asyncio.Future.result() 는 시간초과를 지원하지 않음. yield from 사용 Future 객체의 상태를 가져옴

concurrent.future.as_completed() 함수를 사용해 Future 객체를 볼수 있음


In [4]:
def download_many(cc_list):
    cc_list = cc_list[:5]
    with futures.ThreadPoolExecutor(max_workers=3) as executor:
        to_do = []
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc)
            to_do.append(future)
            msg = 'Scheduled for {}: {}'
            print(msg.format(cc, future))
        
        results = []
        for future in futures.as_completed(to_do): 
            res = future.result()
            msg = '{} result: {!r}'
            print(msg.format(future, res)) # 실제 future 객체를 볼 수 있음 
            results.append(res)
    return len(results)

concurrent.futures 는 Global Interpreter Lock (GIL) 에 의해 제한됨. 병렬로 동작하지 않는다. 

### Blocking I/O and the GIL

CPython 인터프리터는 GIL 을 가짐. GIL 은 한번에 한 스레드만 파이썬 바이트코드를 실행하도록 제한. 단일 파이썬 프로세스가 동시에 다중 CPU 코어를 사용할수 없음.

블로킹 입출력을 실행하는 모든 표준 라이브러리 함수는 OS 에서 결과를 기다리는 동안 GIL 을 해제

입출력 위주의 작업을 실행하는 파이썬 프로그램은 파이썬 구현해도 스레드를 이용.

예) 파이썬 스레드가 네트워크로부터의 응답을 기다리는 동안, 블로킹된 입출력 함수가 GIL 을 해제함으로써 다른 스레드가 실행될 수 있음.


### Launching Processes with concurrent.futures

concurrent.futures [concurrent.futures — Launching parallel tasks](https://docs.python.org/3/library/concurrent.futures.html)

ProcessPoolExecutor 클래스를 사용해서 작업을 여러 파이썬 프로세스에 분산시켜 병렬 컴퓨팅 진행

ProcessPoolExecutor 는 GIL 을 우회함, 계산위주의 작업을 수행해야하는 경우 가용한 CPU 를 모두 사용 

ProcessPoolExecutor 와 ThreadFoolExecutor 는 모두 범용 Executor 인터페이스를 구현, 따라서, concurrent.futures 를 사용하는 경우에는 스레드 기반의 프로그램을 프로세스 기반의 프로그램으로 쉽게 변환가능 



### Experimenting with Executor.map

Excutor.map() 을 이용하면 콜러블을 아주 간단히 동작시에 실행할 수 있다. 


In [1]:
from time import sleep, strftime
from concurrent import futures

def display(*args):
    print(strftime('[%H:%M:%S]'), end=' ')
    print(*args)

def loiter(n):
    msg = '{}loiter({}): doing nothing for {}s...'
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = '{}loiter({}): done.'
    display(msg.format('\t'*n, n))
    return n * 10

def main():
    display('Script starting.')
    executor = futures.ThreadPoolExecutor(max_workers=3)
    results = executor.map(loiter, range(5))
    display('results:', results) # .
    display('Waiting for individual results:')
    for i, result in enumerate(results):
        display('result {}: {}'.format(i, result))
#         for 루프 안에서 enumerate()를 호출하면 암묵적으로 next(result)를 호출하는데, 
#         next(results) 는 먼저 내부적으로 첫번째 호출한 loiter(0) 을 나타내는 Future 객체 _f 의 result() 메서드를 호출한다. 
#         _f.result() 메서드느 _f 가 완료될 떄까지 블로킹되므로, 다음번 결과가 나올때까지는 이 루프는 블로킹된다. 

In [2]:
main()

[14:07:56] Script starting.
[14:07:56][14:07:56]  loiter(0): doing nothing for 0s...
[14:07:56]	loiter(1): doing nothing for 1s...
 loiter(0): done.
[14:07:56] 		loiter(2): doing nothing for 2s...
[14:07:56][14:07:56] 			loiter(3): doing nothing for 3s...
 results: <generator object Executor.map.<locals>.result_iterator at 0x7f3848439eb8>
[14:07:56] Waiting for individual results:
[14:07:56] result 0: 0
[14:07:57] 	loiter(1): done.
[14:07:57] 				loiter(4): doing nothing for 4s...
[14:07:57] result 1: 10
[14:07:58] 		loiter(2): done.
[14:07:58] result 2: 20
[14:07:59] 			loiter(3): done.
[14:07:59] result 3: 30
[14:08:01] 				loiter(4): done.
[14:08:01] result 4: 40


Excutor.map() 은 사용하기 쉽지만, 호출한 순서대로 그대로 결과를 반환하는 특징이 있다. 최종결과를 얻기까지 기다려야 한다. 

완료되는대로 결과를 가져오려면 excutor.submit() 메서드와 futures.as_completed() 함수를 함꼐 사용해야한다. 

submit() 이 다양한 콜러블과 인수를 제출할 수 있는 반면 
executor.map() 은 여러인수에 동일한 콜러블을 실행하도록 설계되어 있으므로, 
excutor.submit()/futures.as_complemted() 조합이 executor.map() 보다 융통성이 높다. 

게다가 일부 ThreadPoolExecutor 객체에서, 다른 일부는 ProcessPoolExecutor 객체에서 가져오는 등 여러 excutor 에서 가져온 Future 객체의 집합을 futures.as_completed() 에 전달 할수 있다. 

### Downloads with Progress Display and Error Handling

text 기반 progress bar [TQDM](https://github.com/tqdm/tqdm)

모두 파일을 실제로 받는 download_one() 함수가 404 에러를 처리한다. 

그외에 다른 에러는 위로 전달되어 download_many() 함수가 처리한다.


### Error Handling in the flags2 Examples

스레드 버전으로 만든 flags_threadpool 

future.as_completed(), futures.ThreadPoolExecutor 사용

In [5]:
def get_flag(base_url, cc):
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    resp = requests.get(url)
    if resp.status_code != 200:
        # 200 에러가 아니면 에러 발생시킴
        resp.raise_for_status() 
    return resp.content

def download_one(cc, base_url, verbose=False):
    try:
        image = get_flag(base_url, cc)
    except requests.exceptions.HTTPError as exc:
        # 404 에러 처리
        res = exc.response
        if res.status_code == 404:
            status = HTTPStatus.not_found
            msg = 'not found'
        else:
            # 그 외의 HTTPError 예외를 다시 발생시켜 호출자로 전달
            raise
    else:
        save_flag(image, cc.lower() + '.gif')
        status = HTTPStatus.ok
        msg = 'OK'
    
    if verbose:
        # -v 옵션을 설정한경우 국가 코드와 메시지가 출력
        print(cc, msg)
    # download_one() 에서 반환한 Result nametuple 은 status 필드에 HTTPStatus.not_found 나 HTTPStatus.ok 를 가짐 
    return Result(status, cc)

download_many 함수는 진행상황을 보여주고 에러를 처리하고, 내려받은 항목의 합계를 구한다. 

In [7]:
def download_many(cc_list, base_url, verbose, max_req):
    counter = collections.Counter() # HTTPStatus ok, not_found, error 합계를 구함
    cc_iter = sorted(cc_list)
    
    if not verbose:
        cc_iter = tqdm.tqdm(cc_iter)
        # tqdm() 은 움직이는 진행막대를 보여주면서 cc_iter 에 들어 있는 항목을 생성하는 반복자를 반환
    
    for cc in cc_iter:
        try:
            res = download_one(cc, base_url, verbose)
            
        except requests.exceptions.HTTPError as exc:
            # HTTP 관련 예외 
            error_msg = 'HTTP error {res.status_code} - {res.reason}'
            error_msg = error_msg.format(res=exc.response)
            
        except requests.exceptions.ConnectionError as exc:
            # 다른 네트워트 에러 
            error_msg = 'Connection error'
        
        else:
            # 아무 예외도 나오지 않은 경우 download_one 반환한 HTTPStatus nametuple 에서 status 값을 가져옴
            error_msg = ''
            status = res.status
            
        if error_msg:
            status = HTTPStatus.error
        counter[status] += 1
            
        if verbose and error_msg:
            print('*** Error for {}: {}'.format(cc, error_msg))
            
    return counter

### Using futures.as_completed


In [14]:
# import collections
# from concurrent import futures
# import requests
# import tqdm
# from flags2_common import main, HTTPStatus
# from flags2_sequential import download_one

DEFAULT_CONCUR_REQ = 30 # -m 옵션이 없으면 최대 동시 요청수 (스레드풀의 크기로 사용)
MAX_CONCUR_REQ = 1000 # -m 에 상관없이 동시 요청수를 제한 (안전장치)

def download_many(cc_list, base_url, verbose, concur_req):
    counter = collections.Counter()
    with futures.ThreadPoolExecutor(max_workers=concur_req) as executor:
        # MAX_CONCUR_REQ, cc_list 길이, -m 중 작은 값을 concur_req 값으로 max_worker 설정 
        to_do_map = {}
        for cc in sorted(cc_list):
            # 결과는 HTTP 응답 시간에 의해 결정
            # concur_req 로 지정한 스레드 풀의 크기가 len(cc_list) 의 항목수보다 적으면 알파넷 순으로 나옴 
            future = executor.submit(download_one, cc, base_url, verbose) # cc 는 콜러블 나머지 뒤 인수는 콜러블에 전달
            to_do_map[future] = cc
            done_iter = futures.as_completed(to_do_map) # 완료된 순서대로 Future 객체를 생성하는 반복자를 반환
        
        if not verbose:
            # 상태 메시지가 아닌경우 진행막대 출력 total 인수를 알려줘야 예상시간 계산 가능
            done_iter = tqdm.tqdm(done_iter, total=len(cc_list))
        
        for future in done_iter:
            try:
                res = future.result()
            except requests.exceptions.HTTPError as exc:
                error_msg = 'HTTP {res.status_code} - {res.reason}'
                error_msg = error_msg.format(res=exc.response)
            except requests.exceptions.ConnectionError as exc:
                error_msg = 'Connection error'
            else:
                error_msg = ''
                status = res.status
            
            if error_msg:
                status = HTTPStatus.error
            counter[status] += 1
            if verbose and error_msg:
                # Future 객체를 키로 사용해서 에러 메시지 정보 제공 
                cc = to_do_map[future]
                print('*** Error for {}: {}'.format(cc, error_msg))
    return counter

# if __name__ == '__main__':
#     main(download_many, DEFAULT_CONCUR_REQ, MAX_CONCUR_REQ)

### Threading and Multiprocessing Alternatives

concurrent.futures 는 단지 스레드를 사용하기 위해 나중에 추가된 방법 

파이썬3에서는 스레드 사용을 중단했다. 

[threading](https://docs.python.org/3/library/threading.html) 모듈을 권장한다. 

futures.ThreadPoolExecutor 로 처리하기 어려운 작업을 수행하는 경우 Thread, Lock, Semaphore 등 threading 모듈의 기본 컴포넌트로 처리 가능하다.

스레드간 데이터 전송은 queue 모듈에서 제공하는 스레드 안전큐를 사용

계산위주의 작업은 여러프로세스를 실행하는 GIL 을 피해야한다. 

[multiprocessing](https://docs.python.org/3/library/multiprocessing.html?highlight=multiprocessing#module-multiprocessing) 패키지를 사용해야 한다. 
